# **Assignment Solution**

In [16]:
import sqlite3
import pandas as pd

# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create a simplified 'Country' table, now with Continent, Population, GNP and LifeExpectancy columns
cursor.execute('''
CREATE TABLE Country (
    Code TEXT PRIMARY KEY,
    Name TEXT,
    Continent TEXT,
    Population INTEGER,
    GNP REAL,
    LifeExpectancy REAL -- Added LifeExpectancy column
);
''')

# Create a simplified 'City' table, now with Population column
cursor.execute('''
CREATE TABLE City (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    CountryCode TEXT,
    Population INTEGER, -- Added Population column for cities
    FOREIGN KEY (CountryCode) REFERENCES Country(Code)
);
''')

# Create a simplified 'Language' table
cursor.execute('''
CREATE TABLE Language (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT UNIQUE
);
''')

# Create a linking table for Country and Language, with an 'IsOfficial' flag
cursor.execute('''
CREATE TABLE CountryLanguage (
    CountryCode TEXT,
    LanguageID INTEGER,
    IsOfficial BOOLEAN,
    PRIMARY KEY (CountryCode, LanguageID),
    FOREIGN KEY (CountryCode) REFERENCES Country(Code),
    FOREIGN KEY (LanguageID) REFERENCES Language(ID)
);
''')

# Insert sample data into Country table, now including continent, population, GNP, and LifeExpectancy
countries_data = [
    ('USA', 'United States', 'North America', 330000000, 23300.0, 78.5),
    ('CAN', 'Canada', 'North America', 38000000, 1900.0, 82.2),
    ('MEX', 'Mexico', 'North America', 128000000, 1200.0, 75.1),
    ('BRA', 'Brazil', 'South America', 215000000, 1800.0, 75.9),
    ('ARG', 'Argentina', 'South America', 45000000, 450.0, 76.5),
    ('COL', 'Colombia', 'South America', 51000000, 300.0, 77.0),
    ('PER', 'Peru', 'South America', 33000000, 220.0, 76.0),
    ('CHN', 'China', 'Asia', 1400000000, 17700.0, 77.4),
    ('IND', 'India', 'Asia', 1380000000, 3200.0, 69.8),
    ('JPN', 'Japan', 'Asia', 126000000, 5000.0, 84.7),
    ('KOR', 'South Korea', 'Asia', 51000000, 1800.0, 83.5),
    ('AUS', 'Australia', 'Oceania', 26000000, 1700.0, 82.9),
    ('NZL', 'New Zealand', 'Oceania', 5000000, 250.0, 82.3),
    ('DEU', 'Germany', 'Europe', 83000000, 4200.0, 80.9),
    ('FRA', 'France', 'Europe', 67000000, 2900.0, 82.5),
    ('GBR', 'United Kingdom', 'Europe', 67000000, 3100.0, 81.3),
    ('ITA', 'Italy', 'Europe', 59000000, 2100.0, 83.2),
    ('ESP', 'Spain', 'Europe', 47000000, 1400.0, 83.3)
]
cursor.executemany('INSERT INTO Country (Code, Name, Continent, Population, GNP, LifeExpectancy) VALUES (?, ?, ?, ?, ?, ?);', countries_data)

# Insert sample data into City table, now including population
cities_data = [
    ('New York', 'USA', 8400000), ('Los Angeles', 'USA', 3900000), ('Chicago', 'USA', 2700000),
    ('Toronto', 'CAN', 2900000), ('Vancouver', 'CAN', 675000),
    ('Mexico City', 'MEX', 9200000), ('Guadalajara', 'MEX', 1400000),
    ('Rio de Janeiro', 'BRA', 6700000), ('Sao Paulo', 'BRA', 12300000), ('Brasilia', 'BRA', 3000000),
    ('Buenos Aires', 'ARG', 2900000),
    ('Beijing', 'CHN', 21500000), ('Shanghai', 'CHN', 26300000), ('Chongqing', 'CHN', 32000000), ('Tianjin', 'CHN', 15400000),
    ('Mumbai', 'IND', 20700000), ('Delhi', 'IND', 32900000),
    ('Sydney', 'AUS', 5300000), ('Melbourne', 'AUS', 5000000),
    ('Berlin', 'DEU', 3700000), ('Munich', 'DEU', 1500000),
    ('Paris', 'FRA', 2100000), ('Marseille', 'FRA', 860000)
]
cursor.executemany('INSERT INTO City (Name, CountryCode, Population) VALUES (?, ?, ?);', cities_data)

# Insert sample data into Language table
languages_data = [
    ('English',), ('Spanish',), ('Portuguese',), ('Mandarin',), ('Hindi',),
    ('German',), ('French',), ('Italian',), ('Japanese',), ('Korean',)
]
cursor.executemany('INSERT INTO Language (Name) VALUES (?);', languages_data)

# Get Language IDs
cursor.execute('SELECT ID, Name FROM Language;')
language_map = {name: id for id, name in cursor.fetchall()}

# Insert sample data into CountryLanguage table
country_languages_data = [
    ('USA', language_map['English'], True), ('USA', language_map['Spanish'], False),
    ('CAN', language_map['English'], True), ('CAN', language_map['French'], True),
    ('MEX', language_map['Spanish'], True),
    ('BRA', language_map['Portuguese'], True),
    ('ARG', language_map['Spanish'], True),
    ('COL', language_map['Spanish'], True),
    ('PER', language_map['Spanish'], True),
    ('CHN', language_map['Mandarin'], True),
    ('IND', language_map['Hindi'], True), ('IND', language_map['English'], True),
    ('JPN', language_map['Japanese'], True),
    ('KOR', language_map['Korean'], True),
    ('AUS', language_map['English'], True),
    ('NZL', language_map['English'], True),
    ('DEU', language_map['German'], True),
    ('FRA', language_map['French'], True),
    ('GBR', language_map['English'], True),
    ('ITA', language_map['Italian'], True),
    ('ESP', language_map['Spanish'], True)
]
cursor.executemany('INSERT INTO CountryLanguage (CountryCode, LanguageID, IsOfficial) VALUES (?, ?, ?);', country_languages_data)

conn.commit()

print("Sample 'world' database created with Country, City, Language, and CountryLanguage tables, including city population.")

Sample 'world' database created with Country, City, Language, and CountryLanguage tables, including city population.


# Question 1: **Count how many cities are there in each country?**

In [2]:
# SQL query to count cities in each country
query = """
SELECT
    C.Name AS CountryName,
    COUNT(CT.ID) AS NumberOfCities
FROM
    Country AS C
LEFT JOIN
    City AS CT ON C.Code = CT.CountryCode
GROUP BY
    C.Name
ORDER BY
    NumberOfCities DESC, CountryName;
"""

# Execute the query and fetch results into a pandas DataFrame
df_result = pd.read_sql_query(query, conn)

# Display the results
display(df_result)

# Close the connection (optional, as it's in-memory)
# conn.close()

,CountryName,NumberOfCities
0,China,4
1,Brazil,3
2,United States,3
3,Australia,2
4,Canada,2
5,France,2
6,Germany,2
7,India,2
8,Mexico,2
9,Argentina,1


# **Question 2 : Display all continents having more than 30 countries.**

In [4]:
# SQL query to display continents having more than a certain number of countries
# (Using '> 3' as a demonstration due to limited sample data, original request was '> 30')
query_continent_count = """
SELECT
    Continent,
    COUNT(Code) AS NumberOfCountries
FROM
    Country
GROUP BY
    Continent
HAVING
    COUNT(Code) > 3; -- Change to '> 30' for the original question with a larger dataset
"""

# Execute the query and fetch results into a pandas DataFrame
df_continents = pd.read_sql_query(query_continent_count, conn)

# Display the results
display(df_continents)

,Continent,NumberOfCountries
0,Asia,4
1,Europe,5
2,South America,4


# **Question 3 : List regions whose total population exceeds 200 million.**


In [6]:
# SQL query to list continents whose total population exceeds 200 million
query_population_by_continent = """
SELECT
    Continent,
    SUM(Population) AS TotalPopulation
FROM
    Country
GROUP BY
    Continent
HAVING
    SUM(Population) > 200000000; -- 200 million
"""

# Execute the query and fetch results into a pandas DataFrame
df_continent_populations = pd.read_sql_query(query_population_by_continent, conn)

# Display the results
display(df_continent_populations)

,Continent,TotalPopulation
0,Asia,2957000000
1,Europe,323000000
2,North America,496000000
3,South America,344000000


# **Question 4 : Find the top 5 continents by average GNP per country.**

In [8]:
# SQL query to find the top 5 continents by average GNP per country
query_avg_gnp_by_continent = """
SELECT
    Continent,
    AVG(GNP) AS AverageGNP
FROM
    Country
WHERE
    GNP IS NOT NULL -- Exclude countries with no GNP data if any
GROUP BY
    Continent
ORDER BY
    AverageGNP DESC
LIMIT 5;
"""

# Execute the query and fetch results into a pandas DataFrame
df_top_5_gnp = pd.read_sql_query(query_avg_gnp_by_continent, conn)

# Display the results
display(df_top_5_gnp)

,Continent,AverageGNP
0,North America,8800.0
1,Asia,6925.0
2,Europe,2740.0
3,Oceania,975.0
4,South America,692.5


# **Question 5 : Find the total number of official languages spoken in each continent.**

In [11]:
# SQL query to find the total number of official languages spoken in each continent
query_official_languages_by_continent = """
SELECT
    C.Continent,
    COUNT(DISTINCT L.Name) AS NumberOfOfficialLanguages
FROM
    Country AS C
JOIN
    CountryLanguage AS CL ON C.Code = CL.CountryCode
JOIN
    Language AS L ON CL.LanguageID = L.ID
WHERE
    CL.IsOfficial = TRUE
GROUP BY
    C.Continent
ORDER BY
    NumberOfOfficialLanguages DESC;
"""

# Execute the query and fetch results into a pandas DataFrame
df_official_languages = pd.read_sql_query(query_official_languages_by_continent, conn)

# Display the results
display(df_official_languages)

,Continent,NumberOfOfficialLanguages
0,Europe,5
1,Asia,5
2,North America,3
3,South America,2
4,Oceania,1


# **Question 6 : Find the maximum and minimum GNP for each continent.**

In [12]:
# SQL query to find the maximum and minimum GNP for each continent
query_min_max_gnp_by_continent = """
SELECT
    Continent,
    MAX(GNP) AS MaximumGNP,
    MIN(GNP) AS MinimumGNP
FROM
    Country
WHERE
    GNP IS NOT NULL -- Exclude countries with no GNP data if any
GROUP BY
    Continent
ORDER BY
    Continent;
"""

# Execute the query and fetch results into a pandas DataFrame
df_min_max_gnp = pd.read_sql_query(query_min_max_gnp_by_continent, conn)

# Display the results
display(df_min_max_gnp)

,Continent,MaximumGNP,MinimumGNP
0,Asia,17700.0,1800.0
1,Europe,4200.0,1400.0
2,North America,23300.0,1200.0
3,Oceania,1700.0,250.0
4,South America,1800.0,220.0


# **Question 7 : Find the country with the highest average city population.**

In [14]:
# SQL query to find the country with the highest average city population
query_highest_avg_city_pop = """
SELECT
    C.Name AS CountryName,
    AVG(CT.Population) AS AverageCityPopulation
FROM
    Country AS C
JOIN
    City AS CT ON C.Code = CT.CountryCode
GROUP BY
    C.Name
ORDER BY
    AverageCityPopulation DESC
LIMIT 1;
"""

# Execute the query and fetch results into a pandas DataFrame
df_highest_avg_city_pop = pd.read_sql_query(query_highest_avg_city_pop, conn)

# Display the results
display(df_highest_avg_city_pop)

,CountryName,AverageCityPopulation
0,India,26800000.0


# **Question 8 : List continents where the average city population is greater than 200,000.**

In [15]:
# SQL query to list continents where the average city population is greater than 200,000
query_avg_city_pop_by_continent = """
SELECT
    C.Continent,
    AVG(CT.Population) AS AverageCityPopulation
FROM
    Country AS C
JOIN
    City AS CT ON C.Code = CT.CountryCode
GROUP BY
    C.Continent
HAVING
    AVG(CT.Population) > 200000;
"""

# Execute the query and fetch results into a pandas DataFrame
df_avg_city_pop_continent = pd.read_sql_query(query_avg_city_pop_by_continent, conn)

# Display the results
display(df_avg_city_pop_continent)

,Continent,AverageCityPopulation
0,Asia,2.480000e+07
1,Europe,2.040000e+06
2,North America,4.167857e+06
3,Oceania,5.150000e+06
4,South America,6.225000e+06


# **Question 9 : Find the total population and average life expectancy for each continent, ordered by average life expectancy descending.**



In [17]:
# SQL query to find the total population and average life expectancy for each continent, ordered by average life expectancy descending
query_population_life_expectancy_by_continent = """
SELECT
    Continent,
    SUM(Population) AS TotalPopulation,
    AVG(LifeExpectancy) AS AverageLifeExpectancy
FROM
    Country
WHERE
    LifeExpectancy IS NOT NULL -- Exclude countries with no LifeExpectancy data if any
GROUP BY
    Continent
ORDER BY
    AverageLifeExpectancy DESC;
"""

# Execute the query and fetch results into a pandas DataFrame
df_population_life_expectancy = pd.read_sql_query(query_population_life_expectancy_by_continent, conn)

# Display the results
display(df_population_life_expectancy)

,Continent,TotalPopulation,AverageLifeExpectancy
0,Oceania,31000000,82.60
1,Europe,323000000,82.24
2,Asia,2957000000,78.85
3,North America,496000000,78.60
4,South America,344000000,76.35


# **Question 10 : Find the top 3 continents with the highest average life expectancy, but only include those where the total population is over 200 million.**

In [18]:
# SQL query to find the top 3 continents with the highest average life expectancy,
# but only include those where the total population is over 200 million.
query_top_continents_life_expectancy = """
SELECT
    Continent,
    SUM(Population) AS TotalPopulation,
    AVG(LifeExpectancy) AS AverageLifeExpectancy
FROM
    Country
WHERE
    LifeExpectancy IS NOT NULL -- Exclude countries with no LifeExpectancy data
GROUP BY
    Continent
HAVING
    SUM(Population) > 200000000 -- Filter for total population over 200 million
ORDER BY
    AverageLifeExpectancy DESC
LIMIT 3;
"""

# Execute the query and fetch results into a pandas DataFrame
df_top_continents_life_expectancy = pd.read_sql_query(query_top_continents_life_expectancy, conn)

# Display the results
display(df_top_continents_life_expectancy)

,Continent,TotalPopulation,AverageLifeExpectancy
0,Europe,323000000,82.24
1,Asia,2957000000,78.85
2,North America,496000000,78.60
